In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install torchcsprng==0.1.3+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1.6MB 672kB/s 


In [6]:
!pip install opacus

     |████████████████████████████████| 92kB 6.7MB/s 


In [7]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
import os
import opacus
from opacus import PrivacyEngine

In [10]:
# __all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
#            'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
#            'wide_resnet50_2', 'wide_resnet101_2']


# model_urls = {
#     'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
#     'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
#     'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
#     'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
#     'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
#     'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
#     'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
#     'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
#     'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
# }

In [11]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, k, kernel_size = 5, stride = 1):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels*k, kernel_size, stride)
        self.tanh = nn.Tanh()
        self.pool = nn.AvgPool2d(kernel_size = 2)
    
    def forward(self, x):
        x = self.conv(x)
        out = self.pool(self.tanh(x))
        
        return out 

### Vanilla LeNet

In [12]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

In [13]:
from torchsummary import summary
model = LeNet5(10)
model.cuda()
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             Tanh-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/ba

### Wide LeNet with Linear Bottle Neck

In [14]:
class Linear_WideLeNet5(nn.Module):

    def __init__(self, n_classes):
        super(Linear_WideLeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=32, kernel_size=5, stride=1, padding = 1),
            nn.Conv2d(in_channels=32, out_channels=8, kernel_size=1, stride=1, padding = 1),
            nn.Conv2d(in_channels=8, out_channels=62, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=62, out_channels=11, kernel_size=1, stride=1, padding = 0),
            nn.Conv2d(in_channels=11, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        # print(x.shape)
        x = torch.flatten(x, 1)
        # print(x.shape)
        logits = self.classifier(x)
        return logits

In [15]:
from torchsummary import summary
Linear_WideLeNet5 = Linear_WideLeNet5(10)
Linear_WideLeNet5.cuda()
summary(Linear_WideLeNet5, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           4,832
            Conv2d-5            [-1, 8, 14, 14]             264
            Conv2d-6           [-1, 62, 10, 10]          12,462
              Tanh-7           [-1, 62, 10, 10]               0
         AvgPool2d-8             [-1, 62, 5, 5]               0
            Conv2d-9             [-1, 11, 5, 5]             693
           Conv2d-10            [-1, 120, 1, 1]          33,120
             Tanh-11            [-1, 120, 1, 1]               0
           Linear-12                   [-1, 84]          10,164
             Tanh-13                   [-1, 84]               0
           Linear-14                   

### Wide LeNet with Non-Linear Bottle Neck

In [16]:
class Non_Linear_WideLeNet5(nn.Module):

    def __init__(self, n_classes):
        super(Non_Linear_WideLeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=32, kernel_size=5, stride=1, padding = 1),
            nn.Tanh(),
            nn.Conv2d(in_channels=32, out_channels=8, kernel_size=1, stride=1, padding = 1),
            nn.Tanh(),
            nn.Conv2d(in_channels=8, out_channels=62, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=62, out_channels=11, kernel_size=1, stride=1, padding = 0),
            nn.Tanh(),
            nn.Conv2d(in_channels=11, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        # print(x.shape)
        x = torch.flatten(x, 1)
        # print(x.shape)
        logits = self.classifier(x)
        return logits

In [17]:
from torchsummary import summary
Non_Linear_WideLeNet5 = Non_Linear_WideLeNet5(10)
Non_Linear_WideLeNet5.cuda()
summary(Non_Linear_WideLeNet5, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           4,832
              Tanh-5           [-1, 32, 12, 12]               0
            Conv2d-6            [-1, 8, 14, 14]             264
              Tanh-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 62, 10, 10]          12,462
              Tanh-9           [-1, 62, 10, 10]               0
        AvgPool2d-10             [-1, 62, 5, 5]               0
           Conv2d-11             [-1, 11, 5, 5]             693
             Tanh-12             [-1, 11, 5, 5]               0
           Conv2d-13            [-1, 120, 1, 1]          33,120
             Tanh-14            [-1, 12

In [18]:
best_acc = []
best_train_acc = best_test_acc = start_epoch = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
optimizer1 = optim.Adam(Linear_WideLeNet5.parameters(), lr = 0.001)

In [19]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [20]:
def train(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [21]:
def test(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(f'Epoch {epoch} Step {batch_idx}/{len(testloader)}', 'Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_test_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/content/drive/My Drive/checkpoint'):
            os.mkdir('/content/drive/My Drive/checkpoint')
        torch.save(state, '/content/drive/My Drive/checkpoint/ckpt.pth')
        best_test_acc = acc

In [23]:
def train_wide(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer1.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer1.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [26]:
def train_NLwide(model, epoch):
    global best_train_acc, best_test_acc
    device = 'cuda'
    model.to(device)
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer2.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer2.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(f'Epoch {epoch} Step {batch_idx}/{len(trainloader)}', 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        acc = 100.*correct/total
        if acc>best_train_acc:
            best_train_acc = acc

In [24]:
batch_size = sample_size = 32

privacy_engine = PrivacyEngine(
    model,
    batch_size,
    sample_size,
    alphas=[10, 100],
    noise_multiplier= 1.3,
    max_grad_norm=1.0,
)
privacy_engine.attach(optimizer)

for epoch in range(start_epoch, start_epoch+50):
    train(model, epoch)
    test(model, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Epoch 0 Step 0/1563 Loss: 2.303 | Acc: 9.375% (3/32)
Epoch 0 Step 1/1563 Loss: 2.300 | Acc: 10.938% (7/64)
Epoch 0 Step 2/1563 Loss: 2.301 | Acc: 11.458% (11/96)
Epoch 0 Step 3/1563 Loss: 2.304 | Acc: 10.938% (14/128)
Epoch 0 Step 4/1563 Loss: 2.304 | Acc: 9.375% (15/160)
Epoch 0 Step 5/1563 Loss: 2.305 | Acc: 8.854% (17/192)
Epoch 0 Step 6/1563 Loss: 2.305 | Acc: 8.482% (19/224)
Epoch 0 Step 7/1563 Loss: 2.304 | Acc: 7.812% (20/256)
Epoch 0 Step 8/1563 Loss: 2.303 | Acc: 7.639% (22/288)
Epoch 0 Step 9/1563 Loss: 2.305 | Acc: 7.500% (24/320)
Epoch 0 Step 10/1563 Loss: 2.305 | Acc: 7.670% (27/352)
Epoch 0 Step 11/1563 Loss: 2.305 | Acc: 7.292% (28/384)
Epoch 0 Step 12/1563 Loss: 2.305 | Acc: 7.452% (31/416)
Epoch 0 Step 13/1563 Loss: 2.304 | Acc: 7.143% (32/448)
Epoch 0 Step 14/1563 Loss: 2.305 | Acc: 7.083% (34/480)
Epoch 0 Step 15/1563 Loss: 2.304 | Acc: 6.836% (35/512)
Epoch 0 Step 16/1563 Loss: 2.304 | Acc: 7.169% (39/544)
Epoch 0 Step 17/1563 Loss: 2.304 | Acc: 7.465% (43/576)
Epoc

/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:283: UserWarning: PrivacyEngine expected a batch of size 32 but the last step received a batch of size 16. This means that the privacy analysis will be a bit more pessimistic. You can set `drop_last = True` in your PyTorch dataloader to avoid this problem completely
  f"PrivacyEngine expected a batch of size {self.batch_size} "


Streaming output truncated to the last 5000 lines.
Epoch 47 Step 632/1563 Loss: 2.046 | Acc: 31.675% (6416/20256)
Epoch 47 Step 633/1563 Loss: 2.046 | Acc: 31.669% (6425/20288)
Epoch 47 Step 634/1563 Loss: 2.046 | Acc: 31.668% (6435/20320)
Epoch 47 Step 635/1563 Loss: 2.045 | Acc: 31.658% (6443/20352)
Epoch 47 Step 636/1563 Loss: 2.045 | Acc: 31.652% (6452/20384)
Epoch 47 Step 637/1563 Loss: 2.046 | Acc: 31.637% (6459/20416)
Epoch 47 Step 638/1563 Loss: 2.046 | Acc: 31.641% (6470/20448)
Epoch 47 Step 639/1563 Loss: 2.046 | Acc: 31.646% (6481/20480)
Epoch 47 Step 640/1563 Loss: 2.046 | Acc: 31.650% (6492/20512)
Epoch 47 Step 641/1563 Loss: 2.046 | Acc: 31.659% (6504/20544)
Epoch 47 Step 642/1563 Loss: 2.046 | Acc: 31.644% (6511/20576)
Epoch 47 Step 643/1563 Loss: 2.046 | Acc: 31.658% (6524/20608)
Epoch 47 Step 644/1563 Loss: 2.046 | Acc: 31.652% (6533/20640)
Epoch 47 Step 645/1563 Loss: 2.046 | Acc: 31.647% (6542/20672)
Epoch 47 Step 646/1563 Loss: 2.046 | Acc: 31.670% (6557/20704)
Epoc

In [25]:
# Training Wide_ResNet16
best_train_acc = best_test_acc = 0
privacy_engine = PrivacyEngine(
    Linear_WideLeNet5,
    batch_size,
    sample_size,
    alphas=[10, 100],
    noise_multiplier=1.3,
    max_grad_norm=1.0,
)
privacy_engine.attach(optimizer1)

for epoch in range(start_epoch, start_epoch+50):
  train_wide(Linear_WideLeNet5, epoch)
  test(Linear_WideLeNet5, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Epoch 0 Step 0/1563 Loss: 2.318 | Acc: 12.500% (4/32)
Epoch 0 Step 1/1563 Loss: 2.312 | Acc: 15.625% (10/64)
Epoch 0 Step 2/1563 Loss: 2.314 | Acc: 13.542% (13/96)
Epoch 0 Step 3/1563 Loss: 2.315 | Acc: 12.500% (16/128)
Epoch 0 Step 4/1563 Loss: 2.315 | Acc: 11.250% (18/160)
Epoch 0 Step 5/1563 Loss: 2.315 | Acc: 10.417% (20/192)
Epoch 0 Step 6/1563 Loss: 2.314 | Acc: 9.821% (22/224)
Epoch 0 Step 7/1563 Loss: 2.312 | Acc: 8.984% (23/256)
Epoch 0 Step 8/1563 Loss: 2.312 | Acc: 8.681% (25/288)
Epoch 0 Step 9/1563 Loss: 2.312 | Acc: 8.750% (28/320)
Epoch 0 Step 10/1563 Loss: 2.311 | Acc: 9.091% (32/352)
Epoch 0 Step 11/1563 Loss: 2.312 | Acc: 8.594% (33/384)
Epoch 0 Step 12/1563 Loss: 2.312 | Acc: 8.173% (34/416)
Epoch 0 Step 13/1563 Loss: 2.310 | Acc: 8.705% (39/448)
Epoch 0 Step 14/1563 Loss: 2.310 | Acc: 8.750% (42/480)
Epoch 0 Step 15/1563 Loss: 2.312 | Acc: 8.203% (42/512)
Epoch 0 Step 16/1563 Loss: 2.312 | Acc: 8.456% (46/544)
Epoch 0 Step 17/1563 Loss: 2.311 | Acc: 8.333% (48/576)


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:283: UserWarning: PrivacyEngine expected a batch of size 32 but the last step received a batch of size 16. This means that the privacy analysis will be a bit more pessimistic. You can set `drop_last = True` in your PyTorch dataloader to avoid this problem completely
  f"PrivacyEngine expected a batch of size {self.batch_size} "


Streaming output truncated to the last 5000 lines.
Epoch 47 Step 632/1563 Loss: 2.109 | Acc: 30.421% (6162/20256)
Epoch 47 Step 633/1563 Loss: 2.109 | Acc: 30.422% (6172/20288)
Epoch 47 Step 634/1563 Loss: 2.109 | Acc: 30.413% (6180/20320)
Epoch 47 Step 635/1563 Loss: 2.109 | Acc: 30.405% (6188/20352)
Epoch 47 Step 636/1563 Loss: 2.110 | Acc: 30.401% (6197/20384)
Epoch 47 Step 637/1563 Loss: 2.109 | Acc: 30.412% (6209/20416)
Epoch 47 Step 638/1563 Loss: 2.109 | Acc: 30.414% (6219/20448)
Epoch 47 Step 639/1563 Loss: 2.108 | Acc: 30.430% (6232/20480)
Epoch 47 Step 640/1563 Loss: 2.108 | Acc: 30.436% (6243/20512)
Epoch 47 Step 641/1563 Loss: 2.108 | Acc: 30.432% (6252/20544)
Epoch 47 Step 642/1563 Loss: 2.108 | Acc: 30.438% (6263/20576)
Epoch 47 Step 643/1563 Loss: 2.108 | Acc: 30.440% (6273/20608)
Epoch 47 Step 644/1563 Loss: 2.108 | Acc: 30.455% (6286/20640)
Epoch 47 Step 645/1563 Loss: 2.108 | Acc: 30.466% (6298/20672)
Epoch 47 Step 646/1563 Loss: 2.108 | Acc: 30.468% (6308/20704)
Epoc

In [29]:
best_train_acc = best_test_acc = 0
optimizer2 = optim.Adam(Non_Linear_WideLeNet5.parameters(), lr = 0.001)
privacy_engine = PrivacyEngine(
    Non_Linear_WideLeNet5,
    batch_size,
    sample_size,
    alphas=[10, 100],
    noise_multiplier=1.3,
    max_grad_norm=1.0,
)
# privacy_engine.attach(optimizer2)

for epoch in range(start_epoch, start_epoch+50):
  train_NLwide(Non_Linear_WideLeNet5, epoch)
  test(Non_Linear_WideLeNet5, epoch)

best_acc.append([best_train_acc, best_test_acc])


Epoch: 0


/usr/local/lib/python3.6/dist-packages/opacus/privacy_engine.py:104: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Streaming output truncated to the last 5000 lines.
Epoch 47 Step 634/1563 Loss: 0.965 | Acc: 66.038% (13419/20320)
Epoch 47 Step 635/1563 Loss: 0.965 | Acc: 66.057% (13444/20352)
Epoch 47 Step 636/1563 Loss: 0.964 | Acc: 66.062% (13466/20384)
Epoch 47 Step 637/1563 Loss: 0.964 | Acc: 66.081% (13491/20416)
Epoch 47 Step 638/1563 Loss: 0.964 | Acc: 66.085% (13513/20448)
Epoch 47 Step 639/1563 Loss: 0.964 | Acc: 66.104% (13538/20480)
Epoch 47 Step 640/1563 Loss: 0.964 | Acc: 66.078% (13554/20512)
Epoch 47 Step 641/1563 Loss: 0.964 | Acc: 66.087% (13577/20544)
Epoch 47 Step 642/1563 Loss: 0.964 | Acc: 66.096% (13600/20576)
Epoch 47 Step 643/1563 Loss: 0.964 | Acc: 66.091% (13620/20608)
Epoch 47 Step 644/1563 Loss: 0.964 | Acc: 66.100% (13643/20640)
Epoch 47 Step 645/1563 Loss: 0.964 | Acc: 66.109% (13666/20672)
Epoch 47 Step 646/1563 Loss: 0.964 | Acc: 66.094% (13684/20704)
Epoch 47 Step 647/1563 Loss: 0.964 | Acc: 66.098% (13706/20736)
Epoch 47 Step 648/1563 Loss: 0.964 | Acc: 66.111% (13

In [30]:
print(best_acc)

[[59.375, 37.03], [56.25, 35.45], [84.375, 67.67]]
